In [1]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
%matplotlib inline

In [2]:
# Load the airbnblistingny.csv dataset.
file_path= "Resources/Airbnblisting_NY.csv"
data = pd.read_csv(file_path, index_col = 0)
data.head()

,host_id,host_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating,zip
id,,,,,,,,,,,,,,,,
2539,2787,9.0,Kensington,40.64529,-73.97238,Private room in rental unit,Private room,2,1.0,1.0,$299.00,30,21,9,4.89,11218
5121,7356,2.0,Bedford-Stuyvesant,40.68535,-73.95512,Private room in rental unit,Private room,2,1.0,1.0,$60.00,30,5,50,4.52,11216
45910,204539,7.0,Ridgewood,40.70309,-73.89963,Entire townhouse,Entire home/apt,16,5.0,10.0,$425.00,30,30,13,4.42,11385
5136,7378,1.0,Sunset Park,40.66265,-73.99454,Entire rental unit,Entire home/apt,4,2.0,2.0,$275.00,21,0,3,5.00,11215
77765,417504,30.0,Greenpoint,40.73777,-73.95366,Room in boutique hotel,Hotel room,2,1.0,1.0,$308.00,2,15,42,4.90,11109


In [3]:
cols = [
    'id',
    'host_id',
    'zip',
    'latitude',
    'longitude',
    'property_type',
    'room_type',
    'accommodates',
    'bedrooms',
    'beds',
    'price',
    'number_of_reviews',
    'review_scores_rating',
    'host_listings_count',
    'availability_30',
    'minimum_nights',
]

data = pd.read_csv('Resources/Airbnblisting_NY.csv', usecols=cols)

In [4]:
# remove NaN values from dataframe
data = data.dropna(how='any', subset=['zip', 'property_type', 'bedrooms', 'beds'])

# drop any inconsistent values
data = data[data['accommodates'] != 0]
data = data[data['bedrooms'] != 0]
data = data[data['beds'] != 0]
data = data[data['price'] != 0.00]

In [5]:
# turn NaN scores with 0 reviews into 'No Reviews'
idx_vals = data['review_scores_rating'][data['number_of_reviews'] == 0].index.values.tolist()
data.loc[idx_vals, ('review_scores_rating')] = data['review_scores_rating'][data['number_of_reviews'] == 0].replace('NaN', 'No Reviews')

# remove inconsistent NaN values
data = data[~data['review_scores_rating'].isnull()]

In [6]:
# convert review_scores_rating into buckets
def convert_scores_buckets(val):
    if val == 'No Reviews':
        return 'No Reviews'
    elif val >= 4.75:
        return '95-100'
    elif val >= 4.50 and val < 4.75:
        return '90-94'
    elif val >= 4.25 and val < 4.50:
        return '85-89'
    elif val >= 4.00 and val < 4.25:
        return '80-84'
    elif val >= 3.50 and val < 4.00:
        return '70-79'
    elif val >= 3.00 and val < 3.50:
        return '60-69'
    elif val >= 2.50 and val < 3.00:
        return '50-59'
    elif val >= 2.00 and val < 2.50:
        return '40-49'
    elif val >= 1.50 and val < 2.00:
        return '30-39'
    elif val >= 1.00 and val < 1.50:
        return '20-29'
    elif val >= 0.50 and val < 1.00:
        return '10-19'
    elif val < 0.50:
        return '0-9'
    
data['review_scores_rating'] = data['review_scores_rating'].apply(convert_scores_buckets)

In [7]:
# convert zipcodes into neighborhoods

dict = {
    '10453': 'Central Bronx',
    '10457': 'Central Bronx',
    '10460': 'Central Bronx',
    '10458': 'Bronx Park and Fordham',
    '10467': 'Bronx Park and Fordham',
    '10468': 'Bronx Park and Fordham',
    '10451': 'High Bridge and Morrisania',
    '10452': 'High Bridge and Morrisania',
    '10456': 'High Bridge and Morrisania',
    '10454': 'Hunts Point and Mott Haven',
    '10455': 'Hunts Point and Mott Haven',
    '10459': 'Hunts Point and Mott Haven',
    '10474': 'Hunts Point and Mott Haven',
    '10045': 'Hunts Point and Mott Haven',
    '10463': 'Kingsbridge and Riverdale',
    '10471': 'Kingsbridge and Riverdale',
    '10550': 'Mount Vernon',
    '10552': 'Mount Vernon',
    '10553': 'Mount Vernon',
    '10705': 'Mount Vernon',
    '10803': 'Mount Vernon',
    '10805': 'Mount Vernon',
    '10466': 'Northeast Bronx',
    '10469': 'Northeast Bronx',
    '10470': 'Northeast Bronx',
    '10475': 'Northeast Bronx',
    '10704': 'Northeast Bronx',
    '10461': 'Southeast Bronx',
    '10462': 'Southeast Bronx',
    '10464': 'Southeast Bronx',
    '10465': 'Southeast Bronx',
    '10472': 'Southeast Bronx',
    '10473': 'Southeast Bronx',
    # Brooklyn Neighborhoods
    '11212': 'Central Brooklyn',
    '11213': 'Central Brooklyn',
    '11216': 'Central Brooklyn',
    '11233': 'Central Brooklyn',
    '11238': 'Central Brooklyn',
    '11209': 'Southwest Brooklyn',
    '11214': 'Southwest Brooklyn',
    '11228': 'Southwest Brooklyn',
    '11204': 'Borough Park',
    '11218': 'Borough Park',
    '11219': 'Borough Park',
    '11230': 'Borough Park',
    '11234': 'Canarsie and Flatlands',
    '11236': 'Canarsie and Flatlands',
    '11239': 'Canarsie and Flatlands',
    '11223': 'Southern Brooklyn',
    '11224': 'Southern Brooklyn',
    '11229': 'Southern Brooklyn',
    '11235': 'Southern Brooklyn',
    '11201': 'Northwest Brooklyn',
    '11205': 'Northwest Brooklyn',
    '11215': 'Northwest Brooklyn',
    '11217': 'Northwest Brooklyn',
    '11231': 'Northwest Brooklyn',
    '11203': 'Flatbush',
    '11210': 'Flatbush',
    '11225': 'Flatbush',
    '11226': 'Flatbush',
    '11126': 'Flatbush',
    '11207': 'East New York and New Lots',
    '11208': 'East New York and New Lots',
    '11211': 'Greenpoint',
    '11222': 'Greenpoint',
    '11220': 'Sunset Park',
    '11232': 'Sunset Park',
    '11206': 'Bushwick and Williamsburg',
    '11221': 'Bushwick and Williamsburg',
    '11237': 'Bushwick and Williamsburg',
    '11249': 'Bushwick and Williamsburg',
    # Staten Island Neighborhoods
    '10302': 'Port Richmond',
    '10303': 'Port Richmond',
    '10310': 'Port Richmond',
    '10306': 'South Shore',
    '10307': 'South Shore',
    '10308': 'South Shore',
    '10309': 'South Shore',
    '10311': 'South Shore',
    '10312': 'South Shore',
    '10301': 'Stapleton and St. George',
    '10304': 'Stapleton and St. George',
    '10305': 'Stapleton and St. George',
    '10314': 'Mid-Island',
    # Manhattan Neighborhoods
    '10026': 'Central Harlem',
    '10027': 'Central Harlem',
    '10030': 'Central Harlem',
    '10037': 'Central Harlem',
    '10039': 'Central Harlem',
    '10001': 'Chelsea and Clinton',
    '10011': 'Chelsea and Clinton',
    '10018': 'Chelsea and Clinton',
    '10019': 'Chelsea and Clinton',
    '10020': 'Chelsea and Clinton',
    '10036': 'Chelsea and Clinton',
    '10029': 'East Harlem',
    '10035': 'East Harlem',
    '10010': 'Gramercy Park and Murray Hill',
    '10016': 'Gramercy Park and Murray Hill',
    '10017': 'Gramercy Park and Murray Hill',
    '10022': 'Gramercy Park and Murray Hill',
    '10012': 'Greenwich Village and Soho',
    '10013': 'Greenwich Village and Soho',
    '10014': 'Greenwich Village and Soho',
    '10004': 'Lower Manhattan',
    '10005': 'Lower Manhattan',
    '10006': 'Lower Manhattan',
    '10007': 'Lower Manhattan',
    '10038': 'Lower Manhattan',
    '10271': 'Lower Manhattan',
    '10280': 'Lower Manhattan',
    '10282': 'Lower Manhattan',
    '10080': 'Lower Manhattan',
    '10281': 'Lower Manhattan',
    '10278': 'Lower Manhattan',
    '10002': 'Lower East Side',
    '10003': 'Lower East Side',
    '10009': 'Lower East Side',
    '10021': 'Upper East Side',
    '10028': 'Upper East Side',
    '10044': 'Upper East Side',
    '10065': 'Upper East Side',
    '10075': 'Upper East Side',
    '10128': 'Upper East Side',
    '10199': 'Upper East Side',
    '10162': 'Upper East Side',
    '10023': 'Upper West Side',
    '10103': 'Upper West Side',
    '10110': 'Upper West Side',
    '10112': 'Upper West Side',
    '10115': 'Upper West Side',
    '10119': 'Upper West Side',
    '10152': 'Upper West Side',
    '10153': 'Upper West Side',
    '10154': 'Upper West Side',
    '10165': 'Upper West Side',
    '10167': 'Upper West Side',
    '10168': 'Upper West Side',
    '10170': 'Upper West Side',
    '10171': 'Upper West Side',
    '10172': 'Upper West Side',
    '10173': 'Upper West Side',
    '10174': 'Upper West Side',
    '10279': 'Upper West Side',
    '10024': 'Upper West Side',
    '10025': 'Upper West Side',
    '10069': 'Upper West Side',
    '10072': 'Upper West Side',
    '10031': 'Inwood and Washington Heights',
    '10032': 'Inwood and Washington Heights',
    '10033': 'Inwood and Washington Heights',
    '10034': 'Inwood and Washington Heights',
    '10040': 'Inwood and Washington Heights',
    # Queens Neighborhoods
    '11361': 'Northeast Queens',
    '11362': 'Northeast Queens',
    '11363': 'Northeast Queens',
    '11364': 'Northeast Queens',
    '11354': 'North Queens',
    '11355': 'North Queens',
    '11356': 'North Queens',
    '11357': 'North Queens',
    '11358': 'North Queens',
    '11359': 'North Queens',
    '11360': 'North Queens',
    '11365': 'Central Queens',
    '11366': 'Central Queens',
    '11367': 'Central Queens',
    '11412': 'Jamaica',
    '11423': 'Jamaica',
    '11424': 'Jamaica',
    '11430': 'Jamaica',
    '11432': 'Jamaica',
    '11433': 'Jamaica',
    '11434': 'Jamaica',
    '11435': 'Jamaica',
    '11436': 'Jamaica',
    '11451': 'Jamaica',
    '11509': 'Jamaica',
    '11101': 'Northwest Queens',
    '11102': 'Northwest Queens',
    '11103': 'Northwest Queens',
    '11104': 'Northwest Queens',
    '11105': 'Northwest Queens',
    '11106': 'Northwest Queens',
    '11107': 'Northwest Queens',
    '11108': 'Northwest Queens',
    '11109': 'Northwest Queens',
    '11374': 'West Central Queens',
    '11375': 'West Central Queens',
    '11379': 'West Central Queens',
    '11385': 'West Central Queens',
    '11580': 'Rockaways',
    '11581': 'Rockaways',
    '11691': 'Rockaways',
    '11692': 'Rockaways',
    '11693': 'Rockaways',
    '11694': 'Rockaways',
    '11695': 'Rockaways',
    '11697': 'Rockaways',
    '11001': 'Southeast Queens',
    '11003': 'Southeast Queens',
    '11004': 'Southeast Queens',
    '11005': 'Southeast Queens',
    '11020': 'Southeast Queens',
    '11021': 'Southeast Queens',
    '11411': 'Southeast Queens',
    '11413': 'Southeast Queens',
    '11422': 'Southeast Queens',
    '11426': 'Southeast Queens',
    '11427': 'Southeast Queens',
    '11428': 'Southeast Queens',
    '11429': 'Southeast Queens',
    '11439': 'Southeast Queens',
    '11414': 'Southwest Queens',
    '11415': 'Southwest Queens',
    '11416': 'Southwest Queens',
    '11417': 'Southwest Queens',
    '11418': 'Southwest Queens',
    '11419': 'Southwest Queens',
    '11420': 'Southwest Queens',
    '11421': 'Southwest Queens',
    '11368': 'West Queens',
    '11369': 'West Queens',
    '11370': 'West Queens',
    '11371': 'West Queens',
    '11372': 'West Queens',
    '11373': 'West Queens',
    '11377': 'West Queens',
    '11378': 'West Queens',
}

def convert_zips_hoods(val):
     return dict[str(val)]
    
data['neighbourhood_cleansed'] = data['zip'].apply(convert_zips_hoods)

In [8]:
data = data.drop('zip', 1)

/var/folders/lt/c6r0brd14djbk6h6mpz3cz0w0000gn/T/ipykernel_54950/2371300133.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('zip', 1)


In [9]:
# encode categorical variables
neighborhood_dummies = pd.get_dummies(data['neighbourhood_cleansed'])
property_dummies = pd.get_dummies(data['property_type'])
room_dummies = pd.get_dummies(data['room_type'])
ratings_scores_dummies = pd.get_dummies(data['review_scores_rating'])

# replace the old columns with our new one-hot encoded ones
df = pd.concat((data.drop(['neighbourhood_cleansed', \
    'property_type', 'room_type', 'review_scores_rating'], axis=1), \
    neighborhood_dummies.astype(str), property_dummies.astype(int), \
    room_dummies.astype(int), ratings_scores_dummies.astype(int)), \
    axis=1)

# move target predictor 'price' to the end of the dataframe
cols = list(df.columns.values)
idx = cols.index('price')
rearrange_cols = cols[:idx] + cols[idx+1:] + [cols[idx]]
df = df[rearrange_cols]

In [10]:
# convert non-categorical variables to floats and standardize
def standardize_col(col):
    mean = np.mean(col)
    std = np.std(col)
    return col.apply(lambda x: (x - mean) / std)

non_cat_vars = ['accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'host_listings_count', 'availability_30', 'minimum_nights']
for col in non_cat_vars:
    df[col] = df[col].astype(float)
    df[col] = standardize_col(df[col])
    
df.head()

,id,host_id,host_listings_count,latitude,longitude,accommodates,bedrooms,beds,minimum_nights,availability_30,...,30-39,40-49,50-59,60-69,70-79,80-84,85-89,90-94,95-100,price
0,2539,2787,-0.061123,40.64529,-73.97238,-0.518969,-0.505571,-0.603634,0.467478,1.803446,...,0,0,0,0,0,0,0,0,1,$299.00
1,5121,7356,-0.097909,40.68535,-73.95512,-0.518969,-0.505571,-0.603634,0.467478,-0.082159,...,0,0,0,0,0,0,0,1,0,$60.00
2,45910,204539,-0.071634,40.70309,-73.89963,6.516032,4.988525,7.101576,0.467478,2.864099,...,0,0,0,0,0,0,1,0,0,$425.00
3,5136,7378,-0.103164,40.66265,-73.99454,0.486031,0.867953,0.252500,0.160028,-0.671410,...,0,0,0,0,0,0,0,0,1,$275.00
4,77765,417504,0.049232,40.73777,-73.95366,-0.518969,-0.505571,-0.603634,-0.489033,1.096345,...,0,0,0,0,0,0,0,0,1,$308.00


In [11]:
# log transform the response 'price'
df['price_log'] = df['price'].str.replace('[\$\,]', '', regex=True)

In [14]:
# read to csv
df.to_csv('Resources/Airbnblisting_NY.csv', index=False)